In [2]:
%cd C:/Users/Mathieu/Desktop/Projets/Benter

C:\Users\Mathieu\Desktop\Projets\Benter


# Train 

- predict the best horse in a given race


### TODO
- try to predict a given distribution according to ranks, instead of the first one
- try to predict best among all combinaisons

In [3]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from utils import winning_validation
from winning_horse_models.sklearn import KNNModel
from winning_horse_models.logistic_regression import LogisticRegressionModel
from utils import training_procedures

C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\l

In [4]:
SOURCE = "PMU"

In [4]:
%%time
sequential_sgd_regression = LogisticRegressionModel()
#sequential_sgd_regression=training_procedures.train_on_each_horse_with_epochs(source=SOURCE, winning_model=sequential_sgd_regression, n_epochs=10, verbose=True)

#sequential_sgd_regression.save_model()

sequential_sgd_regression = LogisticRegressionModel.load_model(trainable=False)

Instructions for updating:
Colocations handled automatically by placer.
Wall time: 109 ms


In [6]:
exact_top_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=sequential_sgd_regression, validation_method=winning_validation.exact_top_k)

C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (35,46,47,61,62,63,64,87) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Mathieu\Desktop\Projets\Benter\utils\preprocess.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

For races w/ 2 horses, 1 races in val, top 1 in right order: 0.000% (Random: 0.000%, Odds 100.000%)
For races w/ 3 horses, 9 races in val, top 1 in right order: 66.667% (Random: 66.667%, Odds 55.556%)
For races w/ 4 horses, 77 races in val, top 1 in right order: 40.260% (Random: 15.584%, Odds 50.649%)
For races w/ 5 horses, 354 races in val, top 1 in right order: 35.876% (Random: 16.949%, Odds 44.350%)
For races w/ 6 horses, 829 races in val, top 1 in right order: 31.846% (Random: 14.596%, Odds 36.188%)
For races w/ 7 horses, 1489 races in val, top 1 in right order: 31.296% (Random: 13.835%, Odds 37.139%)
For races w/ 8 horses, 1914 races in val, top 1 in right order: 26.594% (Random: 12.644%, Odds 35.005%)
For races w/ 9 horses, 2072 races in val, top 1 in right order: 27.268% (Random: 10.714%, Odds 33.687%)
For races w/ 10 horses, 2457 races in val, top 1 in right order: 23.647% (Random: 10.216%, Odds 30.322%)
For races w/ 11 horses, 2269 races in val, top 1 in right order: 22.565% (

In [ ]:
exact_top_5 = winning_validation.compute_validation_error(source=SOURCE, k=5,winning_model=sequential_sgd_regression, validation_method=winning_validation.exact_top_k)

In [ ]:
winning_proba_validation.compute_validation_error(source=SOURCE, k=5, winning_model=sequential_sgd_regression,validation_method=winning_proba_validation.same_top_k)

In [ ]:
winning_proba_validation.compute_validation_error(source=SOURCE, k=5, winning_model=sequential_sgd_regression,validation_method=winning_proba_validation.precision_at_k)

In [ ]:
winning_validation.compute_predicted_proba_on_actual_races(source=SOURCE, k=5,winning_model=sequential_sgd_regression, same_races_support=True)

In [5]:
%%time
from winning_horse_models.xgboost import XGBoostWinningModel
xgboost_winning_model = XGBoostWinningModel.load_model(trainable=False)

#xgboost_winning_model, training_history = training_procedures.train_per_horse(source=SOURCE, winning_model=xgboost_winning_model, verbose=True)
#xgboost_winning_model.save_model()

C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Wall time: 6.54 s


In [6]:
exact_top_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=xgboost_winning_model, validation_method=winning_validation.exact_top_k)

C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (35,46,47,61,62,63,64,87) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


For races w/ 3 horses, 9 races in val, top 1 in right order: 33.333% (Random: 22.222%, Odds 55.556%)
For races w/ 4 horses, 77 races in val, top 1 in right order: 31.169% (Random: 35.065%, Odds 50.649%)
For races w/ 5 horses, 354 races in val, top 1 in right order: 33.616% (Random: 16.949%, Odds 44.350%)
For races w/ 6 horses, 829 races in val, top 1 in right order: 25.935% (Random: 16.888%, Odds 36.188%)
For races w/ 7 horses, 1489 races in val, top 1 in right order: 25.789% (Random: 14.909%, Odds 37.139%)
For races w/ 8 horses, 1914 races in val, top 1 in right order: 25.914% (Random: 11.755%, Odds 35.005%)
For races w/ 9 horses, 2072 races in val, top 1 in right order: 22.153% (Random: 12.403%, Odds 33.687%)
For races w/ 10 horses, 2457 races in val, top 1 in right order: 18.519% (Random: 10.623%, Odds 30.322%)
For races w/ 11 horses, 2269 races in val, top 1 in right order: 18.378% (Random: 9.079%, Odds 28.735%)
For races w/ 12 horses, 2740 races in val, top 1 in right order: 18.32

In [21]:
k=1
source="PMU"
min_horse, max_horse = import_data.get_min_max_horse(source=source)
winning_model=xgboost_winning_model
from winning_horse_models import AbstractWinningModel, ModelNotCreatedOnceError

res={"n_horses_validations":{}}
for n_horses in range(max(k, min_horse), max_horse + 1):

    x_race, rank_race, odds_race = import_data.get_races_per_horse_number(
        y_format="rank",
        x_format="sequential_per_horse",
        n_horses=n_horses,
        on_split="val",
        source=source,
    )
    if x_race.size == 0:
        res["n_horses_validations"][n_horses] = {"n_val_races": 0}
        continue
    try:
        y_hat = winning_model.predict(x=x_race)
    except ModelNotCreatedOnceError:
        res["n_horses_validations"][n_horses] = {"message": "Model was not created"}
        continue

TypeError: 'NoneType' object is not iterable

In [24]:
x_race.shape

(9, 3, 49)

In [27]:
winning_model.get_n_horses_model(n_horses=3).predict(x_race)

ValueError: ('Expecting 2 dimensional numpy.ndarray, got: ', (9, 3, 49))

In [17]:
from xgboost import XGBClassifier
import os
from constants import SAVED_MODELS_DIR
filename = os.listdir(os.path.join(SAVED_MODELS_DIR,cls.__name__))[0]

cls = XGBoostWinningModel

n_horse_model=XGBClassifier()
n_horse_model.load_model(
                    fname=os.path.join(SAVED_MODELS_DIR, cls.__name__, filename)
                )

In [16]:
os.path.join(SAVED_MODELS_DIR, cls.__name__, filename)

'./saved_models\\XGBoostWinningModel\\XGBoostWinningModel_10.json'

In [18]:
n_horse_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='auto', validate_parameters=1, verbosity=None)

In [ ]:
exact_top_1

In [ ]:
winning_proba_validation.compute_validation_error(source=SOURCE, k=5,winning_model=knn_winning_model, validation_method=winning_proba_validation.exact_top_k)

In [ ]:
winning_proba_validation.compute_validation_error(source=SOURCE, k=5, winning_model=knn_winning_model,validation_method=winning_proba_validation.same_top_k)

In [ ]:
winning_proba_validation.compute_validation_error(source=SOURCE, k=5, winning_model=knn_winning_model,validation_method=winning_proba_validation.precision_at_k)

In [ ]:
winning_proba_validation.compute_predicted_proba_on_actual_races(source=SOURCE, k=5,winning_model=knn_winning_model, same_support=True)